# 要約 
このJupyter Notebookは、Chatbot Arenaコンペティションにおける応答の好みを予測するためのモデルを訓練し、その推論を行うことを目的としています。具体的には、GemmaCausalLMという大規模言語モデルを用いて、Loraという手法を利用してモデルの重みを訓練するプロセスが示されています。

### 主な取り組みと手法
1. **データの準備**:
   - Kaggleのデータセットから応答とそれに対するユーザーの選好を取得し、訓練用データセットを構築しています。
   - 特に、テキストデータを前処理し、サロゲート（代用文字）を排除する関数が実装されています。

2. **モデルの設定**:
   - TensorFlowとKerasの環境下で、TPUを利用するための設定が行われています。
   - Kerasの分散処理機能を使い、モデルの重みを複数のTPUに分散するためのデバイスメッシュとレイアウトマップが創出されています。

3. **GemmaCausalLMの利用**:
   - GemmaCausalLMモデルをロードし、Loraを活用して特定の層を訓練不可能に設定し、学習を最適化しています。このアプローチは、モデルの過剰適合を防ぎつつ、効果的な学習を促進します。

4. **モデルの訓練**:
   - モデルアーキテクチャを構築し、訓練データを使ってモデルを訓練します。AdamWオプティマイザーとCategoricalCrossentropy損失を使用して、訓練が行われています。

5. **成果物の保存**:
   - 訓練後、トレーニングの過程で得られたモデルの重みをファイルに保存する処理が含まれています。また、Loraの重みも別に保存され、将来的な推論時に再利用できるようになっています。

### 使用技術とライブラリ
- 使用されているライブラリ: Keras、TensorFlow、JAX、Keras NLP
- 主な関数と手法: TPU環境設定、デバイスメッシュ、Lora訓練、データセットシャッフルなど

このノートブックは、実際のデータセットを用いた実践的な機械学習フォローアップの例であり、ユーザーの好みをモデル化することで、より効果的なチャットボット応答システムの構築を目指しています。

---


# 用語概説 
以下に、初心者がつまずきそうな専門用語の解説を列挙します。特に、実務経験がなく、ドメイン知識が乏しいかもしれないため、注意が必要です。

### 用語解説

1. **エポック (Epoch)**:
   - モデルの学習において、全データセットを一度モデルに通すことを「エポック」と呼びます。1エポックが約3時間かかるとのことから、計算資源やデータ量の多さが学習時間に影響を与えていることがわかります。

2. **Lora (Low-Rank Adaptation)**:
   - モデルの重みを低次元の空間に適応させる手法で、ファインチューニングの際にパラメータ効率を向上させるために使用されます。重みを圧縮することで、必要な計算資源が減少する利点があります。

3. **bfloat16 / float32 / float16**:
   - 異なる浮動小数点形式（数値の表現形式）のことです。bfloat16は機械学習特有のフォーマットで、計算速度向上に寄与しますが、過剰適合を引き起こす可能性があります。float32は32ビットの浮動小数点形式で、一般的に使用される標準的な形式です。float16は16ビットで、メモリ使用量を削減しますが、精度が低いとされます。

4. **TPU (Tensor Processing Unit)**:
   - Googleが開発した専用のハードウェアアクセラレーターで、特にテンソルの計算に最適化されています。深層学習モデルのトレーニングや推論を高速化するために使用されます。

5. **デバイスメッシュ (Device Mesh)**:
   - ディストリビューショントレーニングにおける計算デバイスの配置を指します。特にTPUの場合、モデルを複数のデバイスに効率よく分散させるために必要な構造です。

6. **レイアウトマップ (Layout Map)**:
   - モデルのパラメータがどのデバイスでどのように配置されるかを定義するマッピングです。これにより、重みの分散計算が管理されます。

7. **グローバル平均プーリング (Global Average Pooling)**:
   - 畳み込みニューラルネットワークの出力を平均値で圧縮する手法です。次元を削減し、モデルの過剰適合を防ぐ役割があります。

8. **オプティマイザー (Optimizer)**:
   - モデルのパラメータを更新するためのアルゴリズムです。AdamWはウェイト減衰（Weight Decay）を考慮したAdamオプティマイザーで、トレーニング時の一般化能力を向上させることが期待されます。

9. **重み減衰 (Weight Decay)**:
   - モデルの複雑さをコントロールするための技術で、過剰適合を防ぐためにパラメータの大きさに対してペナルティを課します。通常、ネットワークの重みを小さく保ち、汎化性能を向上させるために使用されます。

10. **トークンID (Token IDs)**:
    - 自然言語処理において、テキストを数値形式に変換することで各単語やトークンを表現します。それぞれの単語がユニークな整数で表されるため、モデルはこれを利用して入力を処理します。

これらの用語は、初学者が機械学習や深層学習の実用的なプロジェクトに取り組む際に理解しておくと良い基礎知識となるでしょう。各用語の背景や応用についてさらに学ぶことで、より深い理解を得ることができます。

---


<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

1 epoch takes ~3hrs 

Just save lora weights and load them in the following notebook for Inference on 2xT4s:

Note: Training on bfloat16 causes model to overfit, so training should be on float32 and inference on float16


https://www.kaggle.com/code/pranshubahadur/inference-tf-gemma-2-9b-lmsys

Inspiration:

https://www.kaggle.com/code/matthewdwatson/gemma-2-fine-tuning-and-inference

https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemma/docs/lora_tuning.ipynb#scrollTo=_Peq7TnLtHse

</div>
<div class="column-right">

# 日本語訳

1エポックは約3時間かかります。

Loraの重みを保存し、次のノートブックで2xT4sの推論に使用してください：

注意：bfloat16での学習はモデルの過剰適合を引き起こすため、学習はfloat32で行い、推論はfloat16で行うべきです。

https://www.kaggle.com/code/pranshubahadur/inference-tf-gemma-2-9b-lmsys

インスピレーション：

https://www.kaggle.com/code/matthewdwatson/gemma-2-fine-tuning-and-inference

https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemma/docs/lora_tuning.ipynb#scrollTo=_Peq7TnLtHse


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install -q -U keras-nlp tensorflow-text
# Install tensorflow-cpu so tensorflow does not attempt to access the TPU.
!pip install -q -U tensorflow-cpu
```

</div>
<div class="column-right">

# 日本語訳

```python
!pip install -q -U keras-nlp tensorflow-text
# tensorflow-cpuをインストールし、tensorflowがTPUにアクセスしないようにします。
!pip install -q -U tensorflow-cpu
```

</div>
</details>

In [ ]:
!pip install -q -U keras-nlp tensorflow-text
# tensorflow-cpuをインストールし、tensorflowがTPUにアクセスしないようにします。
!pip install -q -U tensorflow-cpu

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import jax

jax.devices()
```

</div>
<div class="column-right">

# 日本語訳

```python
import jax

jax.devices()
```

</div>
</details>

In [ ]:
import jax

jax.devices()

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import os

# The Keras 3 distribution API is only implemented for the JAX backend for now
os.environ["KERAS_BACKEND"] = "jax"
# Pre-allocate all TPU memory to minimize memory fragmentation and allocation overhead.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.0"
```

</div>
<div class="column-right">

# 日本語訳

```python
import os

# 現在のところ、Keras 3の配布APIはJAXバックエンドのみに実装されています。
os.environ["KERAS_BACKEND"] = "jax"
# メモリの断片化と割り当てオーバーヘッドを最小限に抑えるために、すべてのTPUメモリを事前に割り当てます。
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.0"
```

</div>
</details>

In [ ]:
import os

# 現在のところ、Keras 3の配布APIはJAXバックエンドのみに実装されています。
os.environ["KERAS_BACKEND"] = "jax"
# メモリの断片化と割り当てオーバーヘッドを最小限に抑えるために、すべてのTPUメモリを事前に割り当てます。
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.0"

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import keras
import keras_nlp
```

</div>
<div class="column-right">

# 日本語訳

```python
import keras
import keras_nlp
```

</div>
</details>

In [ ]:
import keras
import keras_nlp

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Create a device mesh with (1, 8) shape so that the weights are sharded across
# all 8 TPUs.
device_mesh = keras.distribution.DeviceMesh(
    (1, 8),
    ["batch", "model"],
    devices=keras.distribution.list_devices(),
)
```

</div>
<div class="column-right">

# 日本語訳

```python
# 重みがすべての8つのTPUに分散されるように(1, 8)の形状を持つデバイスメッシュを作成します。
device_mesh = keras.distribution.DeviceMesh(
    (1, 8),
    ["batch", "model"],
    devices=keras.distribution.list_devices(),
)
```

</div>
</details>

In [ ]:
# 重みがすべての8つのTPUに分散されるように(1, 8)の形状を持つデバイスメッシュを作成します。
device_mesh = keras.distribution.DeviceMesh(
    (1, 8),
    ["batch", "model"],
    devices=keras.distribution.list_devices(),
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
model_dim = "model"

layout_map = keras.distribution.LayoutMap(device_mesh)

# Weights that match 'token_embedding/embeddings' will be sharded on 8 TPUs
layout_map["token_embedding/embeddings"] = (model_dim, None)
# Regex to match against the query, key and value matrices in attention layers
layout_map["decoder_block.*attention.*(query|key|value)/kernel"] = (model_dim, None, None)
layout_map["decoder_block.*attention_output/kernel"] = (model_dim, None, None)
layout_map["decoder_block.*ffw_gating.*/kernel"] = (None, model_dim)
layout_map["decoder_block.*ffw_linear/kernel"] = (model_dim, None)
```

</div>
<div class="column-right">

# 日本語訳

```python
model_dim = "model"

layout_map = keras.distribution.LayoutMap(device_mesh)

# 'token_embedding/embeddings'に一致する重みは8つのTPUに分散されます。
layout_map["token_embedding/embeddings"] = (model_dim, None)
# 注意レイヤーのクエリ、キー、バリュー行列にマッチする正規表現
layout_map["decoder_block.*attention.*(query|key|value)/kernel"] = (model_dim, None, None)
layout_map["decoder_block.*attention_output/kernel"] = (model_dim, None, None)
layout_map["decoder_block.*ffw_gating.*/kernel"] = (None, model_dim)
layout_map["decoder_block.*ffw_linear/kernel"] = (model_dim, None)
```

</div>
</details>

In [ ]:
model_dim = "model"

layout_map = keras.distribution.LayoutMap(device_mesh)

# 'token_embedding/embeddings'に一致する重みは8つのTPUに分散されます。
layout_map["token_embedding/embeddings"] = (model_dim, None)
# 注意レイヤーのクエリ、キー、バリュー行列にマッチする正規表現
layout_map["decoder_block.*attention.*(query|key|value)/kernel"] = (model_dim, None, None)
layout_map["decoder_block.*attention_output/kernel"] = (model_dim, None, None)
layout_map["decoder_block.*ffw_gating.*/kernel"] = (None, model_dim)
layout_map["decoder_block.*ffw_linear/kernel"] = (model_dim, None)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def remove_surrogates(text):
    return ''.join(char for char in text if not (0xD800 <= ord(char) <= 0xDFFF))

```

</div>
<div class="column-right">

# 日本語訳

```python
def remove_surrogates(text):
    return ''.join(char for char in text if not (0xD800 <= ord(char) <= 0xDFFF))
```

</div>
</details>

In [ ]:
def remove_surrogates(text):
    return ''.join(char for char in text if not (0xD800 <= ord(char) <= 0xDFFF))

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from pandas import read_csv, DataFrame

input_columns = ['prompt', 'response_a', 'response_b']
label_columns = ['winner_model_a', 'winner_model_b', 'winner_tie']

raw_train_dataset = read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
#raw_train_dataset[input_columns] = raw_train_dataset[input_columns].map(lambda x: eval(x)[0])

raw_train_dataset = raw_train_dataset.dropna().drop(['model_a', 'model_b'], axis=1).reset_index(drop=True)


train_dataset = DataFrame({
    'text' : raw_train_dataset[input_columns].agg('\n\nRESPONSE:\n\n'.join, axis=1).apply(lambda x: '\n\nPROMPT\n\n' + x).apply(lambda x: remove_surrogates(x)),
    'label' : raw_train_dataset[label_columns].apply(lambda x: x.values.tolist(), axis=1)
})
```

</div>
<div class="column-right">

# 日本語訳

```python
from pandas import read_csv, DataFrame

input_columns = ['prompt', 'response_a', 'response_b']
label_columns = ['winner_model_a', 'winner_model_b', 'winner_tie']

raw_train_dataset = read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
#raw_train_dataset[input_columns] = raw_train_dataset[input_columns].map(lambda x: eval(x)[0])

raw_train_dataset = raw_train_dataset.dropna().drop(['model_a', 'model_b'], axis=1).reset_index(drop=True)

# データセットを構成します。
train_dataset = DataFrame({
    'text' : raw_train_dataset[input_columns].agg('\n\nRESPONSE:\n\n'.join, axis=1).apply(lambda x: '\n\nPROMPT\n\n' + x).apply(lambda x: remove_surrogates(x)),
    'label' : raw_train_dataset[label_columns].apply(lambda x: x.values.tolist(), axis=1)
})
```

</div>
</details>

In [ ]:
from pandas import read_csv, DataFrame

input_columns = ['prompt', 'response_a', 'response_b']
label_columns = ['winner_model_a', 'winner_model_b', 'winner_tie']

raw_train_dataset = read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
#raw_train_dataset[input_columns] = raw_train_dataset[input_columns].map(lambda x: eval(x)[0])

raw_train_dataset = raw_train_dataset.dropna().drop(['model_a', 'model_b'], axis=1).reset_index(drop=True)

# データセットを構成します。
train_dataset = DataFrame({
    'text' : raw_train_dataset[input_columns].agg('\n\nRESPONSE:\n\n'.join, axis=1).apply(lambda x: '\n\nPROMPT\n\n' + x).apply(lambda x: remove_surrogates(x)),
    'label' : raw_train_dataset[label_columns].apply(lambda x: x.values.tolist(), axis=1)
})

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
model_parallel = keras.distribution.ModelParallel(
    layout_map=layout_map,
    batch_dim_name="batch",
)

keras.distribution.set_distribution(model_parallel)

```

</div>
<div class="column-right">

# 日本語訳

```python
model_parallel = keras.distribution.ModelParallel(
    layout_map=layout_map,
    batch_dim_name="batch",
)

# モデルの並列処理の設定
keras.distribution.set_distribution(model_parallel)
```

</div>
</details>

In [ ]:
model_parallel = keras.distribution.ModelParallel(
    layout_map=layout_map,
    batch_dim_name="batch",
)

# モデルの並列処理の設定
keras.distribution.set_distribution(model_parallel)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
#keras.config.set_floatx("bfloat16")

```

</div>
<div class="column-right">

# 日本語訳

```python
#keras.config.set_floatx("bfloat16")
```

</div>
</details>

In [ ]:
#keras.config.set_floatx("bfloat16")

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("/kaggle/input/gemma2/keras/gemma2_instruct_9b_en/1")
gemma_lm.summary()
```

</div>
<div class="column-right">

# 日本語訳

```python
# GemmaCausalLMモデルを指定されたプリセットから読み込みます。
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("/kaggle/input/gemma2/keras/gemma2_instruct_9b_en/1")
# モデルの概要を表示します。
gemma_lm.summary()
```

</div>
</details>

In [ ]:
# GemmaCausalLMモデルを指定されたプリセットから読み込みます。
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("/kaggle/input/gemma2/keras/gemma2_instruct_9b_en/1")
# モデルの概要を表示します。
gemma_lm.summary()

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
gemma_lm.backbone.enable_lora(rank=16)

```

</div>
<div class="column-right">

# 日本語訳

```python
# Loraを有効にします。
gemma_lm.backbone.enable_lora(rank=16)
```

</div>
</details>

In [ ]:
# Loraを有効にします。
gemma_lm.backbone.enable_lora(rank=16)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
for layer in gemma_lm._backbone.layers[:16]:
    layer.trainable = False
```

</div>
<div class="column-right">

# 日本語訳

```python
# 最初の16層を訓練不可能に設定します。
for layer in gemma_lm._backbone.layers[:16]:
    layer.trainable = False
```

</div>
</details>

In [ ]:
# 最初の16層を訓練不可能に設定します。
for layer in gemma_lm._backbone.layers[:16]:
    layer.trainable = False

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
gemma_lm.summary()
```

</div>
<div class="column-right">

# 日本語訳

```python
# 再度モデルの概要を表示します。
gemma_lm.summary()
```

</div>
</details>

In [ ]:
# 再度モデルの概要を表示します。
gemma_lm.summary()

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def preprocess_fn(text, label=None):
    preprocessed = gemma_lm._preprocessor(text, sequence_length=1024)[0]
    # Ensure the preprocess function returns only the necessary inputs
    return {'token_ids' : preprocessed['token_ids'], 'padding_mask' : preprocessed['padding_mask']}, label if label is not None else text
```

</div>
<div class="column-right">

# 日本語訳

```python
def preprocess_fn(text, label=None):
    preprocessed = gemma_lm._preprocessor(text, sequence_length=1024)[0]
    # 前処理関数が必要な入力のみを返すことを確認します。
    return {'token_ids' : preprocessed['token_ids'], 'padding_mask' : preprocessed['padding_mask']}, label if label is not None else text
```

</div>
</details>

In [ ]:
def preprocess_fn(text, label=None):
    preprocessed = gemma_lm._preprocessor(text, sequence_length=1024)[0]
    # 前処理関数が必要な入力のみを返すことを確認します。
    return {'token_ids' : preprocessed['token_ids'], 'padding_mask' : preprocessed['padding_mask']}, label if label is not None else text

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import tensorflow as tf
from keras.layers import Input, Dense, Flatten, GlobalAveragePooling1D
from keras import Model

inputs = {
    "token_ids": keras.Input(shape=(1024,), dtype=tf.int32, name="token_ids"),
    "padding_mask": keras.Input(shape=(1024,), dtype=tf.int32, name="padding_mask"),
}
x = gemma_lm.backbone(inputs)
print(x.shape)
x = GlobalAveragePooling1D()(x)
print(x.shape)

outputs = Dense(3, 'softmax')(x)
model = Model(inputs, outputs)
```

</div>
<div class="column-right">

# 日本語訳

```python
import tensorflow as tf
from keras.layers import Input, Dense, Flatten, GlobalAveragePooling1D
from keras import Model

# モデルの入力を定義します。
inputs = {
    "token_ids": keras.Input(shape=(1024,), dtype=tf.int32, name="token_ids"),
    "padding_mask": keras.Input(shape=(1024,), dtype=tf.int32, name="padding_mask"),
}
x = gemma_lm.backbone(inputs)
print(x.shape)  # 出力の形状を表示
x = GlobalAveragePooling1D()(x)  # グローバル平均プーリングを適用
print(x.shape)  # 出力の形状を表示

# 出力層を定義します。
outputs = Dense(3, 'softmax')(x)
model = Model(inputs, outputs)
```

</div>
</details>

In [ ]:
import tensorflow as tf
from keras.layers import Input, Dense, Flatten, GlobalAveragePooling1D
from keras import Model

# モデルの入力を定義します。
inputs = {
    "token_ids": keras.Input(shape=(1024,), dtype=tf.int32, name="token_ids"),
    "padding_mask": keras.Input(shape=(1024,), dtype=tf.int32, name="padding_mask"),
}
x = gemma_lm.backbone(inputs)
print(x.shape)  # 出力の形状を表示
x = GlobalAveragePooling1D()(x)  # グローバル平均プーリングを適用
print(x.shape)  # 出力の形状を表示

# 出力層を定義します。
outputs = Dense(3, 'softmax')(x)
model = Model(inputs, outputs)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
optimizer = keras.optimizers.AdamW(
                learning_rate=5e-5,
                weight_decay=0.01,)
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

```

</div>
<div class="column-right">

# 日本語訳

```python
optimizer = keras.optimizers.AdamW(
                learning_rate=5e-5,
                weight_decay=0.01,)
# バイアスとスケールを重み減衰から除外します。
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])
```

</div>
</details>

In [ ]:
optimizer = keras.optimizers.AdamW(
                learning_rate=5e-5,
                weight_decay=0.01,)
# バイアスとスケールを重み減衰から除外します。
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
model.compile(optimizer, loss=tf.keras.losses.CategoricalCrossentropy(),)
```

</div>
<div class="column-right">

# 日本語訳

```python
# モデルをコンパイルします。
model.compile(optimizer, loss=tf.keras.losses.CategoricalCrossentropy(),)
```

</div>
</details>

In [ ]:
# モデルをコンパイルします。
model.compile(optimizer, loss=tf.keras.losses.CategoricalCrossentropy(),)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import tensorflow as tf
ds = tf.data.Dataset.from_tensor_slices((train_dataset.text.values, raw_train_dataset[label_columns].values)).batch(4).map(preprocess_fn)
ds = ds.shuffle(ds.cardinality())

```

</div>
<div class="column-right">

# 日本語訳

```python
import tensorflow as tf
# データセットを作成します。
ds = tf.data.Dataset.from_tensor_slices((train_dataset.text.values, raw_train_dataset[label_columns].values)).batch(4).map(preprocess_fn)
ds = ds.shuffle(ds.cardinality())  # データセットのシャッフル
```

</div>
</details>

In [ ]:
import tensorflow as tf
# データセットを作成します。
ds = tf.data.Dataset.from_tensor_slices((train_dataset.text.values, raw_train_dataset[label_columns].values)).batch(4).map(preprocess_fn)
ds = ds.shuffle(ds.cardinality())  # データセットのシャッフル

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train_split = ds.take(int(len(ds)*0.9))
val_split = ds.skip(int(len(ds)*0.9)).take(int(len(ds)*0.1))
histories = model.fit(train_split, validation_data=[val_split], epochs=1, batch_size=4)
```

</div>
<div class="column-right">

# 日本語訳

```python
# データセットを訓練用と検証用に分割します。
train_split = ds.take(int(len(ds)*0.9))  # 90%を訓練データとして
val_split = ds.skip(int(len(ds)*0.9)).take(int(len(ds)*0.1))  # 残りの10%を検証データとして
# モデルの訓練を実行します。
histories = model.fit(train_split, validation_data=[val_split], epochs=1, batch_size=4)
```

</div>
</details>

In [ ]:
# データセットを訓練用と検証用に分割します。
train_split = ds.take(int(len(ds)*0.9))  # 90%を訓練データとして
val_split = ds.skip(int(len(ds)*0.9)).take(int(len(ds)*0.1))  # 残りの10%を検証データとして
# モデルの訓練を実行します。
histories = model.fit(train_split, validation_data=[val_split], epochs=1, batch_size=4)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import numpy as np
layer = model.get_layer(name='dense')
weights = layer.get_weights()
kernel, bias = weights

# Save the kernel and bias separately
np.save('dense_1_kernel.npy', kernel)
np.save('dense_1_bias.npy', bias)
model.layers[2].save_lora_weights("model.lora.h5")
```

</div>
<div class="column-right">

# 日本語訳

```python
import numpy as np
layer = model.get_layer(name='dense')
weights = layer.get_weights()  # 重みを取得
kernel, bias = weights

# カーネルとバイアスを別々に保存します。
np.save('dense_1_kernel.npy', kernel)
np.save('dense_1_bias.npy', bias)
# Loraの重みを保存します。
model.layers[2].save_lora_weights("model.lora.h5")
```

</div>
</details>

In [ ]:
import numpy as np
layer = model.get_layer(name='dense')
weights = layer.get_weights()  # 重みを取得
kernel, bias = weights

# カーネルとバイアスを別々に保存します。
np.save('dense_1_kernel.npy', kernel)
np.save('dense_1_bias.npy', bias)
# Loraの重みを保存します。
model.layers[2].save_lora_weights("model.lora.h5")